# MPS canonical forms 

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:


import numpy as np
from tensornetwork import ncon
import logging



logging.basicConfig()
logger = logging.getLogger('Something')
logger.setLevel(logging.DEBUG)


import copy 

In [3]:

import myMPSstuff as mps
import myMPOstuff as mpo

from matrixUtils import checkIdMatrix



In [4]:

test2 = mps.myMPS(mps.randMPS(chi=100))
print(f"Entropies=  {test2.getEntropies()}")


Entropies=  [0.0, 4.829998516205387e-05, 1.9779360346658478e-07, 1.736505974043932e-07, 1.126631896853777e-08, 8.500440208979441e-07, 1.6636804517372122e-07, 6.209827349186839e-07, 7.084753982714019e-07, 4.078496228242625e-05, 0.0]


In [5]:

testMPO = mpo.myMPO(mpo.IsingMPO(LL=10, J = 1, g = 0.4))
#print(testMPO.getIndices())

Ising MPO, parameters: J=1 g=0.4,  shapes:
[(1, 3, 2, 2), (3, 3, 2, 2), (3, 3, 2, 2), (3, 3, 2, 2), (3, 3, 2, 2), (3, 3, 2, 2), (3, 3, 2, 2), (3, 3, 2, 2), (3, 3, 2, 2), (3, 1, 2, 2)]


In [6]:

#mpstest = np.random.rand(4,7,3,7)

test = mps.myMPS()

print(test.chis)
print([np.shape(s) for s in test.MPS])

print(test.getNormSlow())
print(test.getNorm())



[1, 20, 20, 20, 20, 20, 20, 1]
[(1, 20, 2), (20, 20, 2), (20, 20, 2), (20, 20, 2), (20, 20, 2), (20, 20, 2), (20, 1, 2)]
(3853171256171380+0.0859375j)
3853171256171380.5


In [7]:
test = mps.myMPS(mps.plusState(10))

print(test.chis)
print([np.shape(s) for s in test.MPS])

# the state should be normalized already 
print(test.getNormSlow())
print(test.getNorm())

test.bringCan()
print(test.getEntropies())  # should be zero for a prod state



[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[(1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2)]
0.9999999999999978
0.9999999999999984
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [8]:
test = mps.myMPS()
print(test.getNormSlow())

print(test.getNorm())
test.bringCan()

(3853171256171380+0.0859375j)
3853171256171380.5


'R'

In [9]:


test3 = mps.myMPS(mps.randMPS(LL=20, chi=200))
print(f"Entropies=  {test3.getEntropies()}")

testMPO = mpo.myMPO()
#print(testMPO.getIndices())



Entropies=  [0.0, 1.0507781804625003e-06, 4.5395212932700594e-08, 6.18394486751068e-08, 2.8159296341644437e-09, 3.1013983482497284e-08, 5.350895276282465e-08, 2.7579413936564797e-08, 1.839786187933455e-08, 4.6922339601826875e-08, 1.6974636176776982e-08, 7.493079711378727e-08, 5.695102935961323e-08, 1.4208561900172424e-08, 6.542566147663851e-08, 2.5545545430923548e-08, 1.301413527306575e-08, 1.0134726710795046e-07, 5.2002173099468896e-08, 1.4958262609650796e-06, 0.0]


## Check that we're actually in proper L-R canon forms by contracting a bit and see if we recover identity matrices and normalizations 

In [10]:
mymps3 = mps.myMPS(mps.randMPS(10,chi=30))

mymps3.bringCan(mode='L',epsTrunc=1e-6)


# So now the MPS should be normalized and in  LEFT canonical form 
# Check by contracting some A's 

print(f"Norm={mymps3.getNorm()}")

mp = mymps3.MPS
sv = mymps3.SV
svinv = mymps3.SVinv


[ checkIdMatrix(ncon([m,np.conj(m)],[[1,-1,2],[1,-2,2]])) for m in mymps3.MPS ]

# Looks good 




Norm=1.0000000000000009
identity, size = 2
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 2
identity, size = 1


[True, True, True, True, True, True, True, True, True, True]

## Try computing an expectation value, first the "hard" way (ncon the whole thing), then using the right-canonical expression

In [11]:
# Try to do it a bit better, define separately psi and psi*
# and compute the norm 

testL = 12
testd = 2
testChi = 8

mps1 = mps.randMPS(testL,chi=testChi, d=testd)

mypsi = mps.myMPS(mps1)
#mypsic = mps.myMPS([np.conj(m) for m in mps1])

# make two copies for later messing around 
mypsic1 = copy.deepcopy(mypsi)
mypsic1conj = mps.myMPS([np.conj(m) for m in mypsic1.MPS])

mypsic2 = copy.deepcopy(mypsi)

print(type(mypsi),type(mypsic1))

# Now define the operator to exp-value,
# say we wanna compute <sigma_x> 
op1 = np.array([[0,1.],[1.,0]])

# at site r = 3 

print("Exp value the hard way: ")

insertjj = 3 
tryind = [[1,3,2]]
tryind.append([1,4,2])

for jj in range(1,testL):
    tj = 3*jj
    if jj == insertjj: # horrible hack 
        tryind.append([tj,tj+3,tj+2])
        tryind.append([tj+2,9999])
        tryind.append([tj+1,tj+4,9999])
    else:
        tryind.append([tj,tj+3,tj+2])
        tryind.append([tj+1,tj+4,tj+2])

tryind[-2][1] = tryind[-1][1]

opList = []
for jj, m in enumerate(mypsi.MPS):
    if jj == insertjj:
        opList.append(m)
        opList.append(op1)
        opList.append(m.conj())
    else:
        opList.append(m)
        opList.append(m.conj())

ev = ncon(opList,tryind)

#print(tryind )
print(np.real_if_close(ev/mypsi.getNormSlow() ))
#print(np.real_if_close(ev/mypsi.getNorm() ))

print("Second way: apply the op to the relevant site, then compute the overlap with itself")
mypsic1.MPS[3] = ncon([op1,mypsic1.MPS[3]],[[-3,1],[-1,-2,1]])
expvaloverlap = mypsic1.overlap(mypsic1conj) 
print(np.real_if_close(expvaloverlap/mypsic1.getNormSlow() ))

print(" Now compute it with the canonical form ")

mypsic2.bringCan(mode='R',epsTrunc=1e-12)

conTen = [np.diag(mypsic2.SV[3]),np.diag(mypsic2.SV[3]),mypsic2.MPS[3],np.conj(mypsic2.MPS[3]),op1]
conIdx = [[1,2],[1,3],[3,5,4],[2,5,6],[4,6]]

[print(np.shape(elem)) for elem in conTen]
print(len(conTen),len(conIdx))

print(np.real_if_close(ncon(conTen,conIdx)))


<class 'myMPSstuff.myMPS'> <class 'myMPSstuff.myMPS'>
Exp value the hard way: 
0.9882075987428638
Second way: apply the op to the relevant site, then compute the overlap with itself
0.9882075987428641
 Now compute it with the canonical form 
(8, 8)
(8, 8)
(8, 8, 2)
(8, 8, 2)
(2, 2)
5 5
0.9882075987428642
